<a href="https://colab.research.google.com/github/josvaldes/trabajoGradoMCD/blob/release%2Fv1.1.0/scrapingColombiaTicFinal3_ipynb_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================================
# NOTEBOOK 1 – Extracción y ETL ColombiaTIC
# Versión: 1.1.0
# Fecha última modificación: 2025-11-05
# Autor: José Valdés
# Descripción: Descarga, normaliza y consolida los boletines trimestrales
# ===============================================================

# ============================================================
#  Versión optimizada: conexión segura, control_cargue, y logs persistentes
# ============================================================

# ===============================================================
# 🧩 BLOQUE 0. Preparación e instalaciones automáticas
# ===============================================================
import sys, subprocess, importlib, os

def pip_install(pkgs):
    for p in pkgs:
        try:
            importlib.import_module(p)
        except ImportError:
            print(f"📦 Instalando {p}...")
            subprocess.run([sys.executable, "-m", "pip", "install", p, "-q"], check=True)

# Paquetes requeridos de Python
pip_install([
    "duckdb",
    "pandas",
    "openpyxl",
    "tqdm",
    "requests",
    "lxml_html_clean",
    "nest_asyncio",
    "playwright",
    "requests-html"
])

# ===============================================================
# 🔧 Instalación del navegador Chromium (si no existe)
# ===============================================================
chromium_path = "/root/.cache/ms-playwright/chromium_headless_shell-1194/chrome-linux/headless_shell"
if not os.path.exists(chromium_path):
    print("⚙️ Chromium no encontrado, instalando...")
    subprocess.run(["playwright", "install", "chromium"], check=True)
else:
    print("✅ Chromium ya disponible.")

# ===============================================================
# 🧩 Instalación de dependencias del sistema para Playwright
# ===============================================================
try:
    print("⚙️ Verificando dependencias del sistema...")
    subprocess.run(["playwright", "install-deps"], check=True)
    print("✅ Dependencias del sistema listas.")
except Exception as e:
    print(f"⚠️ Error instalando dependencias del sistema: {e}")

print("✅ Playwright ya disponible.")
print("✅ Dependencias listas para ejecución del ETL ColombiaTIC")



# ===============================================================
# ========== 1. Configuración especial para Playwright ==========
# ===============================================================
# Instalación del navegador Chromium (una sola vez por entorno)
try:
    from playwright.sync_api import sync_playwright
    print("✅ Playwright ya disponible.")
except Exception:
    print("⚙️ Instalando Playwright Chromium ...")
    subprocess.run(["playwright", "install", "chromium"], check=True)

print("✅ Dependencias listas para ejecución del ETL ColombiaTIC")


# ========== 1. Imports y paths ==========
import os, re, shutil, time, logging
from datetime import datetime
import duckdb
import pandas as pd
import requests
from tqdm import tqdm
import nest_asyncio
nest_asyncio.apply()

from requests_html import AsyncHTMLSession
from google.colab import drive

# Rutas base (ajusta si lo necesitas)
RUTA_EXCEL_DEST   = "/content/gdrive/MyDrive/trabajoGrado/reporte_colombiatic"
RUTA_EXCEL_TEMP   = "/content/gdrive/MyDrive/trabajoGrado/temp_colombiatic"
RUTA_DB_DRIVE_DIR = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos"
RUTA_DB_DRIVE     = os.path.join(RUTA_DB_DRIVE_DIR, "colombiatic.duckdb")
RUTA_DB_LOCAL     = "/content/colombiatic_temp.duckdb"     # base temporal local para evitar IO de Drive
RUTA_LOG          = os.path.join(RUTA_DB_DRIVE_DIR, "colombiatic_etl.log")  # LOG persistente junto a la base

os.makedirs(RUTA_EXCEL_DEST, exist_ok=True)
os.makedirs(RUTA_EXCEL_TEMP, exist_ok=True)
os.makedirs(RUTA_DB_DRIVE_DIR, exist_ok=True)

# ============================================================
# 🔗 Montaje robusto de Google Drive
# ============================================================
from google.colab import drive
import os
import shutil

mount_path = "/content/gdrive"

try:
    # Si ya existe y contiene archivos, lo desmontamos primero
    if os.path.exists(mount_path) and os.listdir(mount_path):
        print("⚙️  Desmontando Drive previo...")
        drive.flush_and_unmount()
        shutil.rmtree(mount_path, ignore_errors=True)

    # Montar Drive limpio
    drive.mount(mount_path, force_remount=True)
    print("✅ Google Drive montado correctamente en:", mount_path)
except Exception as e:
    print(f"⚠️ Error al montar Google Drive: {e}")



# ========== 2. Logger persistente (versión robusta) ==========
import logging, os

# Asegurar ruta del log
os.makedirs(os.path.dirname(RUTA_LOG), exist_ok=True)

logger = logging.getLogger("colombiatic_etl")
logger.setLevel(logging.INFO)

# Limpiar handlers previos (por si se reejecuta)
if logger.hasHandlers():
    logger.handlers.clear()

# Configurar handler de archivo
fh = logging.FileHandler(RUTA_LOG, mode="a", encoding="utf-8")
fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")
fh.setFormatter(fmt)
logger.addHandler(fh)

# ✅ Funciones de log seguras
def log_info(msg):
    try:
        print(msg)
        logger.info(msg)
    except Exception:
        # Si Drive se desconecta, no detiene el proceso
        print(f"[WARN logging deshabilitado temporalmente] {msg}")

def log_warn(msg):
    try:
        print(msg)
        logger.warning(msg)
    except Exception:
        print(f"[WARN logging deshabilitado temporalmente] {msg}")

def log_error(msg):
    try:
        print(msg)
        logger.error(msg)
    except Exception:
        print(f"[WARN logging deshabilitado temporalmente] {msg}")

# Información inicial
log_info("===== INICIO EJECUCIÓN NOTEBOOK 1 (Scraping + ETL) =====")
log_info(f"Carpeta destino Excel: {RUTA_EXCEL_DEST}")
log_info(f"Carpeta temporal Excel: {RUTA_EXCEL_TEMP}")
log_info(f"Base en Drive: {RUTA_DB_DRIVE}")
log_info(f"Log persistente: {RUTA_LOG}")


# ========== 3. Utilidades (nombres y conexión) ==========
def normalizar_nombre(s: str, max_len: int = 60) -> str:
    s = s.lower()
    s = re.sub(r"[^a-z0-9_]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s[:max_len] if max_len else s

def conectar_duckdb_seguro(path_db: str):
    """Conecta o crea base. Si existe en Drive, copia a local y conecta local."""
    # Si existe base en Drive, hacer copia local para trabajar rápido/sin locks
    if os.path.exists(RUTA_DB_DRIVE):
        try:
            if os.path.exists(RUTA_DB_LOCAL):
                os.remove(RUTA_DB_LOCAL)
            shutil.copy2(RUTA_DB_DRIVE, RUTA_DB_LOCAL)
            log_info(f"✅ Copia local creada desde Drive: {RUTA_DB_DRIVE} → {RUTA_DB_LOCAL}")
        except Exception as e:
            log_warn(f"⚠️ No se pudo copiar base desde Drive. Se creará una nueva local. Detalle: {e}")
    # Conectar a la copia local (o nueva si no existía)
    con = duckdb.connect(RUTA_DB_LOCAL)
    log_info(f"💾 Conexión establecida con base local: {RUTA_DB_LOCAL}")
    # Tabla de control (si no existe)
    con.execute("""
    CREATE TABLE IF NOT EXISTS control_cargue (
        archivo TEXT,
        hojas_cargadas INTEGER,
        filas_totales INTEGER,
        fecha_cargue TIMESTAMP,
        estado TEXT
    )
    """)
    log_info("📊 Tabla 'control_cargue' lista.")
    return con

def sincronizar_local_a_drive():
    """Cierra conexión si está abierta y copia local → Drive."""
    try:
        con.close()
    except:
        pass
    try:
        if os.path.exists(RUTA_DB_LOCAL):
            shutil.copy2(RUTA_DB_LOCAL, RUTA_DB_DRIVE)
            log_info(f"📦 Base actualizada copiada de nuevo al Drive: {RUTA_DB_LOCAL} → {RUTA_DB_DRIVE}")
    except Exception as e:
        log_error(f"❌ Error copiando base a Drive: {e}")

# ========== 4. Scraping (encuentra y descarga solo Excel; filtra TIC) ==========
URL_PORTAL = "https://colombiatic.mintic.gov.co/679/w3-channel.html"

# ===============================================================
# ========== 2. Scraping automático con Playwright ==========
# ===============================================================
# ===============================================================
# 🌐 FUNCIÓN: obtener_urls_excel_playwright
# ---------------------------------------------------------------
# Usa Playwright solo para navegar y encontrar los enlaces de Excel (.xls / .xlsx)
# Luego descarga los archivos grandes directamente con requests (streaming)
# ===============================================================

# ===============================================================
# 🌐 FUNCIÓN: obtener_urls_excel_playwright (versión con renombrado automático)
# ===============================================================

import os
import asyncio
from playwright.async_api import async_playwright
import requests
import shutil

async def obtener_urls_excel_playwright(carpeta_destino):
    archivos_descargados = []

    # Diccionario de correspondencia entre boletines y nombre deseado
    renombres = {
        "417629": "reporte_tic_segundo_trimestre_2025.xlsx",
        "407454": "reporte_tic_tercer_trimestre_2024.xlsx",
        "406102": "reporte_tic_segundo_trimestre_2024.xlsx",
        "404030": "reporte_tic_primer_trimestre_2024.xlsx",
        # Puedes ir ampliando esta tabla conforme salgan nuevos boletines
    }

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(accept_downloads=True)
        page = await context.new_page()

        url_principal = "https://colombiatic.mintic.gov.co/679/w3-channel.html"
        log_info(f"🌐 Ingresando a {url_principal} ...")
        await page.goto(url_principal, timeout=120000)

        # Extraer URLs de boletines
        boletines = await page.eval_on_selector_all(
            "a",
            "els => els.map(e => e.href).filter(h => h && h.includes('w3-article'))"
        )
        boletines = list(dict.fromkeys(boletines))  # quitar duplicados
        log_info(f"📰 Boletines detectados: {len(boletines)}")

        for i, href in enumerate(boletines):
            try:
                log_info(f"🔎 Procesando boletín {i+1}: {href}")
                subpage = await context.new_page()
                await subpage.goto(href, timeout=90000)

                # Buscar enlaces Excel directamente
                links = await subpage.eval_on_selector_all(
                    "a",
                    "elements => elements.map(el => el.href).filter(h => h && (h.endsWith('.xls') || h.endsWith('.xlsx')))"
                )

                if links:
                    for enlace in links:
                        nombre_original = enlace.split("/")[-1]
                        destino_temp = os.path.join(carpeta_destino, nombre_original)

                        if not os.path.exists(destino_temp):
                            log_info(f"⬇️ Descargando: {nombre_original}")
                            try:
                                # Descarga directa con streaming
                                with requests.get(enlace, stream=True, timeout=600) as r:
                                    r.raise_for_status()
                                    with open(destino_temp, "wb") as f:
                                        for chunk in r.iter_content(chunk_size=8192):
                                            if chunk:
                                                f.write(chunk)
                                log_info(f"📥 Guardado correctamente: {destino_temp}")
                            except Exception as ex:
                                log_warn(f"⚠️ Error descargando {nombre_original}: {ex}")
                                continue
                        else:
                            log_info(f"⏭️ Ya existe: {destino_temp}")

                        # Intentar renombrar según el diccionario
                        nuevo_nombre = None
                        for clave, nombre_final in renombres.items():
                            if clave in href or clave in nombre_original:
                                nuevo_nombre = nombre_final
                                break

                        if nuevo_nombre:
                            destino_final = os.path.join(carpeta_destino, nuevo_nombre)
                            if destino_temp != destino_final:
                                try:
                                    shutil.move(destino_temp, destino_final)
                                    log_info(f"📝 Renombrado a: {nuevo_nombre}")
                                    archivos_descargados.append(destino_final)
                                except Exception as ex:
                                    log_warn(f"⚠️ Error renombrando {nombre_original} → {nuevo_nombre}: {ex}")
                        else:
                            archivos_descargados.append(destino_temp)
                            log_warn(f"⚠️ No se encontró nombre asignado para {nombre_original}, se conserva nombre original.")

                else:
                    log_warn(f"⚠️ No se encontraron enlaces .xlsx en {href}")

                await subpage.close()

            except Exception as ex:
                log_warn(f"⚠️ Error procesando boletín {i+1}: {ex}")
                continue

        await browser.close()

    log_info(f"✅ Total archivos descargados y renombrados: {len(archivos_descargados)}")
    return archivos_descargados







def descargar_excel(url: str, destino_dir: str) -> str | None:
    """Descarga un Excel y lo nombra a partir del slug del artículo o filename; retorna ruta destino."""
    try:
        resp = requests.get(url, timeout=60)
        if resp.status_code != 200:
            log_warn(f"⚠️ No se pudo descargar {url} (status {resp.status_code})")
            return None
        # nombre base por defecto
        base = url.split("/")[-1].replace(".xlsx", "")
        base = normalizar_nombre(base, 80)

        # guardar en temp
        nombre = f"{base}.xlsx"
        ruta = os.path.join(destino_dir, nombre)
        with open(ruta, "wb") as f:
            f.write(resp.content)
        return ruta
    except Exception as e:
        log_warn(f"⚠️ Error descargando {url}: {e}")
        return None


# Parchear pyppeteer para modo headless estable en Colab
import nest_asyncio, pyppeteer, asyncio
nest_asyncio.apply()

async def fix_pyppeteer():
    browser = await pyppeteer.launch(
        headless=True,
        args=[
            "--no-sandbox",
            "--disable-gpu",
            "--disable-dev-shm-usage",
            "--disable-setuid-sandbox",
            "--disable-extensions",
            "--disable-infobars",
            "--window-size=1920,1080",
        ]
    )
    await browser.close()

asyncio.get_event_loop().run_until_complete(fix_pyppeteer())
print("✅ pyppeteer inicializado correctamente en modo headless.")

# ===============================================================
# 🧩 Instalar dependencias del sistema necesarias para Playwright
# ===============================================================
import subprocess

print("⚙️ Instalando dependencias del sistema para Playwright (solo una vez)...")
subprocess.run(["apt-get", "install", "-y",
                "libatk1.0-0",
                "libatk-bridge2.0-0",
                "libatspi2.0-0",
                "libxcomposite1",
                "libxdamage1",
                "libxfixes3",
                "libxrandr2",
                "libgbm1",
                "libxkbcommon0",
                "libpango-1.0-0",
                "libasound2",
                "libgtk-3-0"], check=True)

print("✅ Dependencias del sistema instaladas correctamente.")



# Lanzar scraping
log_info("🌐 Iniciando scraping de boletines...")
try:
    # Ejecutar asincrónicamente en Colab
    import asyncio
    urls_xlsx = asyncio.get_event_loop().run_until_complete(obtener_urls_excel_playwright(RUTA_EXCEL_DEST))
except RuntimeError:
    # Loop ya corriendo: usar nest_asyncio y crear uno nuevo
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    urls_xlsx = loop.run_until_complete(obtener_urls_excel_playwright())

log_info(f"📊 Archivos Excel detectados (únicos): {len(urls_xlsx)}")

# Descarga a carpeta temporal
descargados = []
for u in tqdm(urls_xlsx, desc="⬇️ Descargando archivos Excel"):
    ruta_tmp = descargar_excel(u, RUTA_EXCEL_TEMP)
    if ruta_tmp:
        descargados.append(ruta_tmp)

log_info(f"📦 Archivos descargados en temp: {len(descargados)}")

# Detectar TIC y mover a carpeta final. Borrar los no TIC
movidos = []
eliminados = []
for ruta in descargados:
    nombre = os.path.basename(ruta)
    # Heurística: debe contener "tic" en el nombre (en muchos casos funciona),
    # y/o es el archivo principal que el portal publica (ya trae "sector_tic" o similar).
    if "tic" in nombre.lower():
        destino = os.path.join(RUTA_EXCEL_DEST, nombre)
        if os.path.exists(destino):
            os.remove(destino)
        shutil.move(ruta, destino)
        movidos.append(destino)
        log_info(f"📦 Movido a carpeta destino: {os.path.basename(destino)}")
    else:
        os.remove(ruta)
        eliminados.append(nombre)
        log_info(f"🧹 Eliminado archivo no relevante: {nombre}")

# Limpieza final de temp (si quedó algo)
try:
    for f in os.listdir(RUTA_EXCEL_TEMP):
        try:
            os.remove(os.path.join(RUTA_EXCEL_TEMP, f))
        except:
            pass
except:
    pass

log_info("🎯 Scraping finalizado.")


# 💡 Limpieza interna para evitar crecimiento del archivo DuckDB
import duckdb
try:
    con = duckdb.connect("/content/colombiatic_temp.duckdb")
    con.execute("VACUUM;")  # Compacta y elimina espacio no usado
    con.close()
    log_info("🧹 Base compactada con VACUUM antes de copiar al Drive.")
except Exception as e:
    log_warn(f"⚠️ No se pudo ejecutar VACUUM: {e}")



# ========== 5. Conexión a DuckDB (local) ==========
# Importante: conectar después del scraping
con = conectar_duckdb_seguro(RUTA_DB_DRIVE)



# ========== 6. Carga incremental de Excel TIC ==========
def archivos_tic_actuales():
    return [f for f in os.listdir(RUTA_EXCEL_DEST) if f.lower().endswith((".xlsx", ".xls"))]

# Obtén los ya cargados desde control_cargue
try:
    ya_cargados = set(
        con.execute("SELECT DISTINCT archivo FROM control_cargue").fetchdf()["archivo"].tolist()
    )
except Exception:
    ya_cargados = set()

archivos = archivos_tic_actuales()
pendientes = [f for f in archivos if f not in ya_cargados]

print("\n📂 Archivos disponibles actualmente en carpeta destino:")
for f in archivos:
    print(f"   • {f}")

if not pendientes:
    log_info("⏭️ No hay archivos nuevos para cargar. La base está actualizada.")
else:
    log_info(f"🆕 Archivos TIC nuevos detectados: {len(pendientes)}")

for nombre_archivo in pendientes:
    ruta_archivo = os.path.join(RUTA_EXCEL_DEST, nombre_archivo)
    estado = "OK"
    total_filas = 0
    hojas_cargadas = 0
    log_info(f"\n📘 Procesando: {nombre_archivo}")

    try:
        xls = pd.ExcelFile(ruta_archivo)
        hojas = xls.sheet_names
        # proceso hoja a hoja
        for hoja in hojas:
            try:
                # Detectar encabezado: primeras 10 filas probando
                df_valid = None
                for fila_enc in range(0, 10):
                    df_try = pd.read_excel(ruta_archivo, sheet_name=hoja, header=fila_enc, engine="openpyxl")
                    if df_try.columns.notna().sum() > 2:
                        df_valid = df_try
                        break
                if df_valid is None or df_valid.empty:
                    continue

                df_valid = df_valid.astype(str)

                # nombre de tabla
                base = normalizar_nombre(os.path.splitext(nombre_archivo)[0], 48)
                hoja_n = normalizar_nombre(hoja, 10)  # corto para no pasarse de 63
                nombre_tabla = f"{base}_{hoja_n}"

                # Evitar duplicados de tabla
                tablas_existentes = con.execute("SHOW TABLES").fetchdf()["name"].tolist()
                if nombre_tabla in tablas_existentes:
                    log_info(f"   ⏭️ La tabla '{nombre_tabla}' ya existe, se omite.")
                else:
                    con.register("tmp_df", df_valid)
                    con.execute(f"CREATE TABLE '{nombre_tabla}' AS SELECT * FROM tmp_df")
                    con.unregister("tmp_df")
                    hojas_cargadas += 1
                    total_filas += len(df_valid)
                    log_info(f"   ✅ Hoja '{hoja}' cargada como '{nombre_tabla}' ({len(df_valid)} filas)")
            except Exception as e_hoja:
                estado = f"ERROR_HOJA:{hoja}:{str(e_hoja)[:150]}"
                log_warn(f"⚠️ {estado}")

    except Exception as e_file:
        estado = f"ERROR_ARCHIVO:{str(e_file)[:150]}"
        log_warn(f"⚠️ {estado}")

    # Registrar cargue a nivel archivo (incremental)
    try:
        con.execute("""
            INSERT INTO control_cargue (archivo, hojas_cargadas, filas_totales, fecha_cargue, estado)
            VALUES (?, ?, ?, ?, ?)
        """, [nombre_archivo, hojas_cargadas, total_filas, datetime.now(), estado])
        log_info(f"📝 Registrado en control_cargue: {nombre_archivo} | hojas={hojas_cargadas} | filas={total_filas} | estado={estado}")
    except Exception as e_ins:
        log_warn(f"⚠️ Error insertando en control_cargue: {e_ins}")

# Sincronizar base local a Drive y cerrar
sincronizar_local_a_drive()

# ============================================================
# 🧾 EXPORTAR TABLA DE CONTROL (para trazabilidad del proceso)
# ============================================================
import os
import pandas as pd
import duckdb
from datetime import datetime

# 1️⃣ Parámetro del trimestre procesado
PERIODO_ACTUAL = "2025_T2"  # Ajustar según trimestre a procesar

# 2️⃣ Conectarse a la base ya sincronizada en Drive
db_path = RUTA_DB_DRIVE
con_exp = duckdb.connect(db_path, read_only=True)

# 3️⃣ Leer la tabla de control_cargue
try:
    df_control = con_exp.execute("""
        SELECT archivo, hojas_cargadas, filas_totales, fecha_cargue, estado
        FROM control_cargue
        ORDER BY fecha_cargue DESC
    """).fetchdf()
    print(f"📊 Registros en control_cargue: {len(df_control)}")
except Exception as e:
    print(f"⚠️ No se pudo leer la tabla control_cargue: {e}")
    df_control = pd.DataFrame()

con_exp.close()

# 4️⃣ Crear carpeta destino en Drive
OUT_DIR = f"/content/gdrive/MyDrive/trabajoGrado/resultados/control_cargue/{PERIODO_ACTUAL}"
os.makedirs(OUT_DIR, exist_ok=True)

# 5️⃣ Exportar resultados
csv_path = f"{OUT_DIR}/control_cargue_{PERIODO_ACTUAL}.csv"
parquet_path = f"{OUT_DIR}/control_cargue_{PERIODO_ACTUAL}.parquet"

if not df_control.empty:
    df_control.to_csv(csv_path, index=False)
    df_control.to_parquet(parquet_path, index=False)
    print(f"✅ Archivo exportado correctamente:\n - {csv_path}\n - {parquet_path}")
else:
    print("⚠️ No hay datos para exportar.")

# 6️⃣ Crear mini resumen Markdown (opcional)
md_path = f"{OUT_DIR}/RESUMEN_{PERIODO_ACTUAL}.md"
with open(md_path, "w", encoding="utf-8") as f:
    f.write(f"# Control de Cargue – {PERIODO_ACTUAL}\n\n")
    f.write(f"Fecha de exportación: {datetime.now():%Y-%m-%d %H:%M:%S}\n\n")
    if not df_control.empty:
        ultimos = df_control.head(10)
        f.write(ultimos.to_markdown(index=False))
    else:
        f.write("No se encontraron registros en control_cargue.\n")

print(f"🗂️ Resumen generado en: {md_path}")


# ========== 7. Revisar la base (resumen final) ==========
# Para evitar el error de conexión cerrada, abrimos una conexión **ligera** a la base en Drive solo para consulta
try:
    con_check = duckdb.connect(RUTA_DB_DRIVE, read_only=True)
    tablas = con_check.execute("SHOW TABLES").fetchdf()
    print("\n📂 Tablas en la base:")
    display(tablas)

    control = con_check.execute("""
        SELECT archivo, hojas_cargadas, filas_totales, fecha_cargue, estado
        FROM control_cargue
        ORDER BY fecha_cargue DESC
        LIMIT 20
    """).fetchdf()
    print("\n📋 Últimos registros en control_cargue:")
    display(control)

    total_control = con_check.execute("SELECT COUNT(*) AS total FROM control_cargue").fetchdf()
    print("\n🧮 Total de registros en control_cargue:")
    display(total_control)

    # tamaño del archivo de base
    tam_mb = os.path.getsize(RUTA_DB_DRIVE) / (1024*1024)
    print(f"\n💾 Tamaño actual de la base: {tam_mb:.2f} MB")
    con_check.close()
except Exception as e:
    log_warn(f"⚠️ No se pudo consultar la base final: {e}")

log_info("===== FIN EJECUCIÓN NOTEBOOK 1 =====")
print("\n✅ Proceso completado.")
print(f"🗂 Log del ETL: {RUTA_LOG}")
print(f"🗄️ Base DuckDB: {RUTA_DB_DRIVE}")
print(f"📁 Carpeta Excel (solo TIC): {RUTA_EXCEL_DEST}")


📦 Instalando requests-html...
✅ Chromium ya disponible.
⚙️ Verificando dependencias del sistema...
✅ Dependencias del sistema listas.
✅ Playwright ya disponible.
✅ Dependencias listas para ejecución del ETL ColombiaTIC
✅ Playwright ya disponible.
✅ Dependencias listas para ejecución del ETL ColombiaTIC
⚙️  Desmontando Drive previo...


INFO:colombiatic_etl:===== INICIO EJECUCIÓN NOTEBOOK 1 (Scraping + ETL) =====
INFO:colombiatic_etl:Carpeta destino Excel: /content/gdrive/MyDrive/trabajoGrado/reporte_colombiatic
INFO:colombiatic_etl:Carpeta temporal Excel: /content/gdrive/MyDrive/trabajoGrado/temp_colombiatic
INFO:colombiatic_etl:Base en Drive: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb
INFO:colombiatic_etl:Log persistente: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic_etl.log


Mounted at /content/gdrive
✅ Google Drive montado correctamente en: /content/gdrive
===== INICIO EJECUCIÓN NOTEBOOK 1 (Scraping + ETL) =====
Carpeta destino Excel: /content/gdrive/MyDrive/trabajoGrado/reporte_colombiatic
Carpeta temporal Excel: /content/gdrive/MyDrive/trabajoGrado/temp_colombiatic
Base en Drive: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb
Log persistente: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic_etl.log
✅ pyppeteer inicializado correctamente en modo headless.
⚙️ Instalando dependencias del sistema para Playwright (solo una vez)...


INFO:colombiatic_etl:🌐 Iniciando scraping de boletines...


✅ Dependencias del sistema instaladas correctamente.
🌐 Iniciando scraping de boletines...


...

,archivo,hojas_cargadas,filas_totales,fecha_cargue,estado
0,reporte_tic_segundo_trimestre_2025.xlsx,21,2349811,2025-11-12 20:54:30.836810,"ERROR_HOJA:18:Passed header=[4], len of 1, but..."
1,articles-406102_archivo_xls.xlsx,0,0,2025-11-12 19:50:26.395185,OK
2,articles-407454_archivo_xls.xlsx,0,0,2025-11-12 19:50:24.786946,"ERROR_HOJA:3:Passed header=[4], len of 1, but ..."
3,articles-417629_recurso_1.xlsx,21,2349811,2025-11-12 19:47:47.034690,"ERROR_HOJA:18:Passed header=[4], len of 1, but..."
4,articles_404030_archivo_xls.xlsx,21,2363737,2025-10-23 18:32:13.245150,"ERROR_HOJA:18:Passed header=[4], len of 1, but..."
5,articles_407454_archivo_xls.xlsx,3,864401,2025-10-23 18:26:17.344438,"ERROR_HOJA:3:Passed header=[4], len of 1, but ..."
6,articles_406102_archivo_xls.xlsx,3,11398,2025-10-23 18:23:45.249435,OK
7,reporte_tic_tercer_trimestre_2024.xlsx,22,2364691,2025-10-23 02:24:33.008973,OK
8,reporte_tic_cuarto_trimestre_2024.xlsx,23,2365465,2025-10-23 02:18:42.719431,OK
9,reporte_tic_segundo_trimestre_2024.xlsx,23,2269671,2025-10-23 02:12:54.000494,OK



🧮 Total de registros en control_cargue:


,total
0,23


INFO:colombiatic_etl:===== FIN EJECUCIÓN NOTEBOOK 1 =====



💾 Tamaño actual de la base: 854.51 MB
===== FIN EJECUCIÓN NOTEBOOK 1 =====

✅ Proceso completado.
🗂 Log del ETL: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic_etl.log
🗄️ Base DuckDB: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb
📁 Carpeta Excel (solo TIC): /content/gdrive/MyDrive/trabajoGrado/reporte_colombiatic


In [ ]:
import duckdb, pandas as pd

# Conexión (solo lectura)
db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path, read_only=True)

# Listar todas las tablas que contengan "_4_1"
tablas_41 = con.execute("""
    SELECT table_name AS name
    FROM duckdb_tables()
    WHERE table_name LIKE '%4_1%'
    ORDER BY table_name
""").fetchdf()

print("📋 Tablas 4_1 detectadas:")
display(tablas_41)



📋 Tablas 4_1 detectadas:


,name
0,articles_404030_archivo_xls_4_1
1,articles_417629_recurso_1_4_1
2,consolidado_tic_4_1
3,consolidado_tic_4_1_filtrado
4,consolidado_tic_4_1_limpia
5,reporte_tic_cuarto_trimestre_2024_1
6,reporte_tic_cuarto_trimestre_2024_10
7,reporte_tic_cuarto_trimestre_2024_11
8,reporte_tic_cuarto_trimestre_2024_12
9,reporte_tic_cuarto_trimestre_2024_13


Bloque de código final para consolidar automáticamente

In [ ]:
import duckdb, pandas as pd

# --- Asegurar conexión limpia ---
try:
    con.close()
except:
    pass

db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path)
print("✅ Conectado a la base correctamente.\n")

# --- Buscar todas las tablas que terminen en _4_1 ---
tablas_41 = con.execute("""
    SELECT table_name AS name
    FROM duckdb_tables()
    WHERE table_name LIKE '%4_1'
    ORDER BY table_name
""").fetchdf()

if tablas_41.empty:
    print("⚠️ No hay tablas 4.1 disponibles.")
else:
    print(f"📋 Tablas detectadas para consolidar ({len(tablas_41)}):")
    display(tablas_41)

    # --- Obtener todas las columnas únicas entre las tablas 4.1 ---
    columnas_union = set()
    columnas_por_tabla = {}

    for t in tablas_41["name"]:
        cols = con.execute(f"PRAGMA table_info('{t}')").fetchdf()["name"].tolist()
        columnas_union.update(cols)
        columnas_por_tabla[t] = cols

    columnas_union = sorted(list(columnas_union))
    print(f"🧱 Total columnas únicas detectadas: {len(columnas_union)}\n")

    # --- Crear sentencias SELECT alineadas ---
    selects = []
    for t, cols in columnas_por_tabla.items():
        # agregar NULL para columnas faltantes
        select_cols = [f"'{t}' AS origen"]
        for col in columnas_union:
            if col in cols:
                select_cols.append(f'"{col}"')
            else:
                select_cols.append(f"NULL AS \"{col}\"")
        selects.append(f"SELECT {', '.join(select_cols)} FROM '{t}'")

    union_query = " UNION ALL ".join(selects)

    # --- Crear tabla consolidada ---
    con.execute(f"""
        CREATE OR REPLACE TABLE consolidado_tic_4_1 AS {union_query}
    """)
    print("✅ Tabla consolidada creada: consolidado_tic_4_1")

    # --- Mostrar resumen de consolidación ---
    resumen = con.execute("""
        SELECT origen, COUNT(*) AS registros
        FROM consolidado_tic_4_1
        GROUP BY origen
        ORDER BY origen
    """).fetchdf()

    print("\n📊 Registros por tabla origen:")
    display(resumen)

    total = con.execute("SELECT COUNT(*) FROM consolidado_tic_4_1").fetchone()[0]
    print(f"🧮 Total de registros consolidados: {total:,}")

con.close()
print("\n🔚 Conexión cerrada correctamente.")


✅ Conectado a la base correctamente.

📋 Tablas detectadas para consolidar (10):


,name
0,articles_404030_archivo_xls_4_1
1,articles_417629_recurso_1_4_1
2,consolidado_tic_4_1
3,reporte_tic_cuarto_trimestre_2024_1
4,reporte_tic_cuarto_trimestre_2024_4_1
5,reporte_tic_segundo_trimestre_2024_1
6,reporte_tic_segundo_trimestre_2024_4_1
7,reporte_tic_segundo_trimestre_2025_4_1
8,reporte_tic_tercer_trimestre_2024_1
9,reporte_tic_tercer_trimestre_2024_4_1


🧱 Total columnas únicas detectadas: 22



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Tabla consolidada creada: consolidado_tic_4_1

📊 Registros por tabla origen:


,origen,registros
0,articles_404030_archivo_xls_4_1,385911
1,articles_417629_recurso_1_4_1,197075
2,consolidado_tic_4_1,18709615
3,reporte_tic_cuarto_trimestre_2024_1,11698
4,reporte_tic_cuarto_trimestre_2024_4_1,571379
5,reporte_tic_segundo_trimestre_2024_1,10657
6,reporte_tic_segundo_trimestre_2024_4_1,104218
7,reporte_tic_segundo_trimestre_2025_4_1,197075
8,reporte_tic_tercer_trimestre_2024_1,11316
9,reporte_tic_tercer_trimestre_2024_4_1,756075


🧮 Total de registros consolidados: 20,955,019

🔚 Conexión cerrada correctamente.


Bloque para explorar y visualizar la tabla consolidada

Código optimizado para limpiar y renombrar consolidado_tic_4_1

In [ ]:
# ============================================================
# 🧠 UNIÓN SEGURA DE TABLAS CON COLUMNAS DIFERENTES
# ============================================================

# 1️⃣ Identificar las columnas clave comunes esperadas
columnas_requeridas = ["AÑO", "TRIMESTRE", "DEPARTAMENTO", "MUNICIPIO", "PROVEEDOR", "TECNOLOGÍA"]

# 2️⃣ Filtrar solo las tablas que contienen todas esas columnas
tablas_validas = []
for t in tablas["table_name"]:
    cols = con.execute(f"""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_name = '{t}'
    """).df()["column_name"].str.upper().tolist()

    if all(c in [col.upper() for col in cols] for c in columnas_requeridas):
        tablas_validas.append(t)

print(f"🧩 Tablas con estructura compatible: {len(tablas_validas)}")
if len(tablas_validas) == 0:
    raise Exception("❌ Ninguna tabla tiene todas las columnas requeridas. Verifica los nombres o limpia las fuentes.")

# 3️⃣ Construir UNION ALL solo con las columnas comunes
query_union_segura = " UNION ALL ".join([
    f"""
    SELECT
        CAST(AÑO AS INTEGER) AS AÑO,
        CAST(TRIMESTRE AS INTEGER) AS TRIMESTRE,
        TRIM(DEPARTAMENTO) AS DEPARTAMENTO,
        TRIM(MUNICIPIO) AS MUNICIPIO,
        TRIM(PROVEEDOR) AS PROVEEDOR,
        TRIM(TECNOLOGÍA) AS TECNOLOGÍA,
        '{t}' AS origen
    FROM {t}
    WHERE TRY_CAST(AÑO AS INTEGER) IS NOT NULL
    """
    for t in tablas_validas
])

# 4️⃣ Crear vista previa con LIMIT (procesado en disco, sin cargar todo)
vista = con.execute(f"""
    {query_union_segura}
    LIMIT 20;
""").df()

print("👀 Vista previa consolidada (20 filas):")
display(vista.head(20))

# 5️⃣ Resumen general de registros por departamento y año
print("\n📊 Resumen por DEPARTAMENTO y AÑO:")
resumen_final = con.execute(f"""
    SELECT
        DEPARTAMENTO,
        AÑO,
        COUNT(DISTINCT MUNICIPIO) AS municipios_unicos,
        COUNT(*) AS total_registros
    FROM ({query_union_segura})
    GROUP BY DEPARTAMENTO, AÑO
    ORDER BY AÑO DESC, total_registros DESC
    LIMIT 20;
""").df()

display(resumen_final)


🧩 Tablas con estructura compatible: 2
👀 Vista previa consolidada (20 filas):


,AÑO,TRIMESTRE,DEPARTAMENTO,MUNICIPIO,PROVEEDOR,TECNOLOGÍA,origen
0,2022,3,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,WIFI,consolidado_tic_4_1_filtrado
1,2022,3,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,WIFI,consolidado_tic_4_1_filtrado
2,2022,3,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,WIFI,consolidado_tic_4_1_filtrado
3,2022,3,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,WIFI,consolidado_tic_4_1_filtrado
4,2022,3,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,WIFI,consolidado_tic_4_1_filtrado
5,2022,3,ANTIOQUIA,AMALFI,@DIGITAL GROUP SAS,FIBER TO THE HOME (FTTH),consolidado_tic_4_1_filtrado
6,2022,3,ANTIOQUIA,AMALFI,@DIGITAL GROUP SAS,FIBER TO THE HOME (FTTH),consolidado_tic_4_1_filtrado
7,2022,3,ANTIOQUIA,AMALFI,@DIGITAL GROUP SAS,FIBER TO THE HOME (FTTH),consolidado_tic_4_1_filtrado
8,2022,3,ANTIOQUIA,AMALFI,@DIGITAL GROUP SAS,FIBER TO THE HOME (FTTH),consolidado_tic_4_1_filtrado
9,2022,3,ANTIOQUIA,AMALFI,@DIGITAL GROUP SAS,FIBER TO THE HOME (FTTH),consolidado_tic_4_1_filtrado



📊 Resumen por DEPARTAMENTO y AÑO:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,DEPARTAMENTO,AÑO,municipios_unicos,total_registros
0,ANTIOQUIA,2022,125,2184976
1,CUNDINAMARCA,2022,116,1911216
2,VALLE DEL CAUCA,2022,42,1367984
3,SANTANDER,2022,87,819008
4,BOYACÁ,2022,123,629424
5,CALDAS,2022,27,511976
6,ATLÁNTICO,2022,23,505608
7,TOLIMA,2022,47,487256
8,RISARALDA,2022,14,484080
9,BOGOTÁ D.C.,2022,1,454920


Validación de cobertura y consistencia

In [ ]:
import duckdb, pandas as pd

db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path, read_only=True)

print("✅ Conectado a la base para análisis exploratorio.\n")

# --- 1️⃣ Departamentos únicos y conteo ---
print("📍 Departamentos y número de municipios reportados:")
deptos = con.execute("""
    SELECT DEPARTAMENTO, COUNT(DISTINCT MUNICIPIO) AS municipios_unicos, COUNT(*) AS total_registros
    FROM consolidado_tic_4_1_limpia
    GROUP BY DEPARTAMENTO
    ORDER BY DEPARTAMENTO
""").fetchdf()
display(deptos)

# --- 2️⃣ Cobertura temporal ---
print("\n🗓️ Años y trimestres disponibles:")
periodos = con.execute("""
    SELECT AÑO, TRIMESTRE, COUNT(*) AS registros
    FROM consolidado_tic_4_1_limpia
    GROUP BY AÑO, TRIMESTRE
    ORDER BY AÑO, TRIMESTRE
""").fetchdf()
display(periodos)

# --- 3️⃣ Validación de velocidades ---
print("\n📈 Rango de velocidades detectadas:")
velocidades = con.execute("""
    SELECT
        MIN("VELOCIDAD SUBIDA") AS min_subida,
        MAX("VELOCIDAD SUBIDA") AS max_subida,
        MIN("VELOCIDAD BAJADA") AS min_bajada,
        MAX("VELOCIDAD BAJADA") AS max_bajada,
        ROUND(AVG("VELOCIDAD SUBIDA"),2) AS promedio_subida,
        ROUND(AVG("VELOCIDAD BAJADA"),2) AS promedio_bajada
    FROM consolidado_tic_4_1_limpia
""").fetchdf()
display(velocidades)

con.close()
print("\n🔚 Conexión cerrada correctamente.")


✅ Conectado a la base para análisis exploratorio.

📍 Departamentos y número de municipios reportados:


,DEPARTAMENTO,municipios_unicos,total_registros
0,AMAZONAS,9,5390
1,ANTIOQUIA,125,1437515
2,ARAUCA,7,48350
3,"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANT...",2,10355
4,ATLÁNTICO,23,332620
5,BOGOTÁ D.C.,1,298635
6,BOLÍVAR,46,255200
7,BOYACÁ,123,422270
8,CALDAS,27,337885
9,CAQUETA,16,66290



🗓️ Años y trimestres disponibles:


,AÑO,TRIMESTRE,registros
0,2021.0,4.0,521065
1,2021.0,NaN,5
2,2022.0,1.0,924320
3,2022.0,2.0,1854545
4,2022.0,3.0,2840780
5,2022.0,4.0,2947105
6,2022.0,NaN,14
7,NaN,NaN,76



📈 Rango de velocidades detectadas:


,min_subida,max_subida,min_bajada,max_bajada,promedio_subida,promedio_bajada
0,0.0,3450300.0,0.0,3450300.0,111.03,161.31



🔚 Conexión cerrada correctamente.


In [ ]:
import duckdb
con = duckdb.connect("/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb")
con.execute("SHOW TABLES").fetchdf()


,name
0,articles_404030_archivo_xls_1
1,articles_404030_archivo_xls_10
2,articles_404030_archivo_xls_11
3,articles_404030_archivo_xls_12
4,articles_404030_archivo_xls_13
...,...
139,reporte_tic_tercer_trimestre_2024_7
140,reporte_tic_tercer_trimestre_2024_8
141,reporte_tic_tercer_trimestre_2024_9
142,reporte_tic_tercer_trimestre_2024_CONTENIDO


Verificación de unicidad

In [ ]:
import duckdb
con = duckdb.connect("/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb")

duplicados = con.execute("""
    SELECT COUNT(*) AS total_registros,
           COUNT(DISTINCT (AÑO, TRIMESTRE, DEPARTAMENTO, MUNICIPIO, PROVEEDOR, "TECNOLOGÍA")) AS registros_unicos
    FROM consolidado_tic_4_1_limpia
""").fetchdf()

print(duplicados)
con.close()


   total_registros  registros_unicos
0          9087910             55982


Consulta para verificar dimensiones y muestra de datos

In [ ]:
import duckdb
import pandas as pd

# --- Conexión a la base ---
db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path)

# --- 1️⃣ Dimensiones de la tabla ---
dim = con.execute("""
    SELECT
        COUNT(*) AS total_registros,
        COUNT(DISTINCT DEPARTAMENTO) AS departamentos,
        COUNT(DISTINCT MUNICIPIO) AS municipios,
        COUNT(DISTINCT AÑO) AS anios,
        COUNT(DISTINCT TRIMESTRE) AS trimestres,
        COUNT(DISTINCT PROVEEDOR) AS proveedores,
        COUNT(DISTINCT "TECNOLOGÍA") AS tecnologias
    FROM consolidado_tic_4_1_limpia
""").fetchdf()

print("📏 Dimensiones de la tabla `consolidado_tic_4_1_limpia`:")
display(dim)

# --- 2️⃣ Vista previa ordenada por año y trimestre ---
muestra = con.execute("""
    SELECT AÑO, TRIMESTRE, DEPARTAMENTO, MUNICIPIO, PROVEEDOR,
           "SEGMENTO", "TECNOLOGÍA", "VELOCIDAD SUBIDA", "VELOCIDAD BAJADA",
           "NO. ACCESOS FIJOS A INTERNET"
    FROM consolidado_tic_4_1_limpia
    WHERE AÑO IS NOT NULL
    ORDER BY AÑO DESC, TRIMESTRE DESC
    LIMIT 20
""").fetchdf()

print("\n👀 Vista previa de registros representativos:")
display(muestra)

con.close()
print("\n🔚 Conexión cerrada correctamente.")


📏 Dimensiones de la tabla `consolidado_tic_4_1_limpia`:


,total_registros,departamentos,municipios,anios,trimestres,proveedores,tecnologias
0,9087910,35,1037,2,4,1085,18



👀 Vista previa de registros representativos:


,AÑO,TRIMESTRE,DEPARTAMENTO,MUNICIPIO,PROVEEDOR,SEGMENTO,TECNOLOGÍA,VELOCIDAD SUBIDA,VELOCIDAD BAJADA,NO. ACCESOS FIJOS A INTERNET
0,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,30.0,60.0,2
1,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,15.0,30.0,1
2,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,5.0,10.0,3
3,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,4.0,8.0,3
4,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,20.0,40.0,1
5,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,RESIDENCIAL - ESTRATO 1,WIFI,2.5,5.0,147
6,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,2.5,5.0,3
7,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,12.5,25.0,3
8,2022.0,4.0,ANTIOQUIA,AMALFI,@DIGITAL GROUP SAS,RESIDENCIAL - ESTRATO 4,FIBER TO THE HOME (FTTH),7.0,15.0,1
9,2022.0,4.0,ANTIOQUIA,AMALFI,@DIGITAL GROUP SAS,RESIDENCIAL - ESTRATO 4,FIBER TO THE HOME (FTTH),6.0,12.0,5



🔚 Conexión cerrada correctamente.


Bloque de detección y limpieza de outliers

In [ ]:
import duckdb
import pandas as pd
import numpy as np

db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path)

# --- Cargar los datos a memoria para depuración ---
df = con.execute("SELECT * FROM consolidado_tic_4_1_limpia").fetchdf()

print(f"📊 Registros iniciales: {len(df):,}")

# --- Convertir a numérico por seguridad ---
for col in ["VELOCIDAD SUBIDA", "VELOCIDAD BAJADA"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# --- Calcular IQR para cada variable ---
def limpiar_outliers_iqr(serie):
    q1 = serie.quantile(0.25)
    q3 = serie.quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr
    return serie.clip(lower=limite_inferior, upper=limite_superior)

df["VELOCIDAD SUBIDA_LIMPIA"] = limpiar_outliers_iqr(df["VELOCIDAD SUBIDA"])
df["VELOCIDAD BAJADA_LIMPIA"] = limpiar_outliers_iqr(df["VELOCIDAD BAJADA"])

# --- Conteo de valores corregidos ---
cambios_subida = (df["VELOCIDAD SUBIDA"] != df["VELOCIDAD SUBIDA_LIMPIA"]).sum()
cambios_bajada = (df["VELOCIDAD BAJADA"] != df["VELOCIDAD BAJADA_LIMPIA"]).sum()

print(f"⚙️ Registros ajustados (subida): {cambios_subida:,}")
print(f"⚙️ Registros ajustados (bajada): {cambios_bajada:,}")

# --- Guardar tabla limpia ---
con.execute("DROP TABLE IF EXISTS consolidado_tic_4_1_filtrado")
con.register("df_temp", df)
con.execute("""
    CREATE TABLE consolidado_tic_4_1_filtrado AS
    SELECT
        origen, AÑO, TRIMESTRE, DEPARTAMENTO, MUNICIPIO, PROVEEDOR,
        SEGMENTO, TECNOLOGÍA,
        "NO. ACCESOS FIJOS A INTERNET",
        "CÓDIGO DANE", "CÓDIGO DANE_1",
        "VELOCIDAD SUBIDA_LIMPIA" AS VELOCIDAD_SUBIDA,
        "VELOCIDAD BAJADA_LIMPIA" AS VELOCIDAD_BAJADA
    FROM df_temp
""")
con.unregister("df_temp")

# --- Verificar dimensiones ---
dim = con.execute("SELECT COUNT(*) AS total, COUNT(DISTINCT MUNICIPIO) AS municipios FROM consolidado_tic_4_1_filtrado").fetchdf()
print("\n✅ Tabla `consolidado_tic_4_1_filtrado` creada correctamente:")
display(dim)

con.close()
print("\n🔚 Conexión cerrada correctamente.")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

📊 Registros iniciales: 9,087,910
⚙️ Registros ajustados (subida): 1,520,190
⚙️ Registros ajustados (bajada): 829,565


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✅ Tabla `consolidado_tic_4_1_filtrado` creada correctamente:


,total,municipios
0,9087910,1037



🔚 Conexión cerrada correctamente.


Validación de duplicados

In [ ]:
# ============================================================
# 🔍 VERIFICACIÓN DE DUPLICADOS EN TABLA CONTROL_CARGUE
# ============================================================

import duckdb
import pandas as pd

# Reutilizamos la ruta de conexión existente
db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"

# Conexión en modo lectura
con_chk = duckdb.connect(db_path, read_only=True)

# Consulta de verificación
dup = con_chk.execute("""
    SELECT archivo, COUNT(*) AS veces
    FROM control_cargue
    GROUP BY archivo
    HAVING veces > 1
""").fetchdf()

con_chk.close()

# Resultado
if dup.empty:
    print("✅ Sin duplicados en control_cargue.")
else:
    print("⚠️ Archivos duplicados detectados:")
    display(dup)


⚠️ Archivos duplicados detectados:


,archivo,veces
0,boletn_trimestral_del_sector_de_tv_por_suscrip...,3
1,boletn_trimestral_del_sector_tic_cifras_primer...,4
2,reporte_tic_segundo_trimestre_2024.xlsx,2
3,boletn_trimestral_del_sector_postal_cifras_seg...,3
4,reporte_tic_cuarto_trimestre_2024.xlsx,2
5,reporte_tic_tercer_trimestre_2024.xlsx,2


limpiar metadatos automáticamente

In [ ]:
import json, os

nb_path = "/content/gdrive/MyDrive/Colab Notebooks/scrapingColombiaTicFinal3_ipynb_clean.ipynb"

if os.path.exists(nb_path):
    with open(nb_path, "r", encoding="utf-8") as f:
        nb = json.load(f)

    if "widgets" in nb.get("metadata", {}):
        nb["metadata"].pop("widgets", None)
        print("🧹 Metadatos de widgets eliminados.")
    else:
        print("✅ No había metadatos de widgets problemáticos.")

    with open(nb_path, "w", encoding="utf-8") as f:
        json.dump(nb, f, ensure_ascii=False, indent=2)
    print(f"Notebook listo para subir: {nb_path}")
else:
    print("⚠️ No se encontró el archivo .ipynb en /content/")


⚠️ No se encontró el archivo .ipynb en /content/


In [ ]:
import os

# Detectar notebook actual
!pwd
!ls -l *.ipynb


/content
ls: cannot access '*.ipynb': No such file or directory
